In [145]:
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd
import random

import time
import datetime
import random

In [146]:
def get_grid(model):
    """ Esta función nos permite obtener el estado de los diferentes agentes.
        *param* model : Modelo del que obtendrá la información. 
        *return* una matriz con la información del estado de cada uno de los agentes."""
    grid = np.zeros( (model.grid.width, model.grid.height) )
    for (content, x, y) in model.grid.coord_iter():
        for contenido in content: 
            if isinstance(contenido, BoxAgent):
                if contenido.state == contenido.IS_BOX:
                    if contenido.stack ==  1:
                        grid[x][y] = 4

                    if contenido.stack ==  2:
                        grid[x][y] = 2
                        
                    if contenido.stack ==  3:
                        grid[x][y] = 3

                    if contenido.stack ==  4:
                        grid[x][y] = 5

                    if contenido.stack ==  5:
                        grid[x][y] = 1
                else:
                    grid[x][y] = 9 #fondo
            if isinstance(contenido, RobotAgent):
                if contenido.has_a_box == 0:
                    grid[x][y] = 6
                else:
                    grid[x][y] = 7
    return grid

In [147]:
""" Este agente es el agente de las cajas, puede ser una caja o simplemente nada (representa el piso)"""
class BoxAgent(Agent):
    IS_BOX = 1
    NOT_BOX = 0
    
    def __init__(self, pos, model, state = NOT_BOX):
        super().__init__(pos, model)
        self.x, self.y = pos
        self.state = state
        self.stack = 0

In [148]:
#""" Este agente es el agente de los robots """
class RobotAgent(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.move_to = None #where to move to
        self.has_a_box = 0 #if the robor has a box
        self.x, self.y = pos #the positon of the robot
    
    def step(self):
        
        #""" Si el robot tiene una caja """
        if self.has_a_box == 0:
            if self.model.cont_boxes != self.model.num_box:
                self.model.cont_steps += 1
                
            neighbours = self.model.grid.get_neighbors(self.pos, moore = False, include_center = True)
            for neighbor in neighbours:
                if neighbor.pos == self.pos:
                    #""" Si el robot no está ensia de una caja seguirá buscando """
                    if neighbor.state == neighbor.NOT_BOX: #NO ES CAJA, SIGUE BUSCANDO
                        neighborhood = self.model.grid.get_neighborhood(self.pos, moore = False, include_center = True)
                        self.move_to = self.random.choice(neighborhood)
                    
                    #""" Si sí es una caja la toma """
                    else:
                        #""" Si la caja en la que está ya ha sido apilada, entonces no la toma y sigue buscando más cajas """
                        if(neighbor.stack == 0):
                            neighbor.state = neighbor.NOT_BOX #ES CAJA, LA TOMA
                            self.has_a_box = 1
                            self.model.cont_boxes += 1
                            self.move_to = self.pos
                        else:
                            neighborhood = self.model.grid.get_neighborhood(self.pos, moore = False, include_center = True)
                            self.move_to = self.random.choice(neighborhood)
                    break
                    
        #""" Si el robot tiene una caja, busca el lugar para dejarla """
        else:
            
            """ Si aún no llega a la posición de la caja """
            if self.model.cont_boxes != self.model.num_box - 1:
                self.model.cont_steps += 1
                
            if self.x != self.model.if_is_full_x:
                self.x -= 1
            
            elif self.x == self.model.if_is_full_x:
                if self.y != self.model.if_is_full_y:
                    if self.y < 0:
                        self.y = self.y * -1
                        self.y = self.y % self.model.width
                        self.y = self.model.width - self.y
                    self.y -= 1
                    
                #""" Si ya llegó a la posición de la caja """    
                elif self.y == self.model.if_is_full_y: #SI YA LLEGARON A LA POSICION PARA DEJAR LAS CAJAS
                     
                    neighbours = self.model.grid.get_neighbors(self.pos, moore = False, include_center = True)
                    for neighbor in neighbours:
                        if isinstance(neighbor, BoxAgent):
                            if neighbor.pos == self.pos: #Checamos que hay ensima del stack                                
                                if neighbor.state == neighbor.NOT_BOX: #Si lo que está ensima NO es una CAJA, la dejamos
                                    neighbor.state = neighbor.IS_BOX #Se pone la caja
                                    neighbor.stack = 1 #Ahora tenemos una caja en la pila
                                    self.has_a_box = 0 #Y le quitamos la caja a nuestro agente

                                    self.x += 1 #Ahora movemos al agente de lugar para poder seguir buscando
                                    
                                else: #Si lo que está ensima SÍ es una CAJA, significa que es un stack
                                    if (neighbor.stack == 4): #Si YA tiene 5 cajas el stack
                                        neighbor.stack += 1 #Le agregamos la última
                                        self.model.if_is_full_y += 1 #Movemos el siguiente lugar  dónde se dejaran cajas
                                        self.has_a_box = 0 #Y le quitamos la caja a nuestro agente
                                        self.x += 1 #Ahora movemos al agente de lugar para poder seguir buscando
                                        
                                        """ Si la primera fila ya se llenó de stacks entonces nos movemos a la siguiente """
                                        if self.model.if_is_full_y == self.model.width - 1:
                                            self.model.if_is_full_x += 1
                                            self.model.if_is_full_y = 0
                                        
                                    else: #Si NO tiene 5 cajas el stack 
                                        neighbor.stack += 1 #Ahora tenemos otra caja en la pila
                                        self.has_a_box = 0 #Y le quitamos la caja a nuestro agente
                                        self.x += 1 #Ahora movemos al agente de lugar para poder seguir buscando
                
                                break
            
            self.move_to = self.x, self.y
                    
        self.model.grid.move_agent(self, self.move_to)
        

In [149]:
""" Este es nuestro modelo dónde todos los agentes interactuan """
class WareHouseModel(Model):
    def __init__(self, height, width, num_box):
        self.num_box = num_box
        self.width = width
        self.num_robots = 5
        self.grid = MultiGrid(height, width, True)
        self.schedule = SimultaneousActivation(self)
        self.if_is_full_y = 0
        self.if_is_full_x = 0
        self.cont_steps = 0
        self.cont_boxes = 1
        
        
        not_boxes = list(self.grid.empties)
        """ Repartimos las celdas que tendrán cajas """
        for cells in range(self.num_box):
            no_box = random.choice(not_boxes)
            box = BoxAgent(no_box, self)
            box.state = box.IS_BOX
            self.grid.place_agent(box, no_box)
            self.schedule.add(box)
            not_boxes.remove(no_box)
            
        not_boxes = list(self.grid.empties)
        """ Repartimos las celdas sin caja.""" 
        for cells in not_boxes:
            free_cells = BoxAgent(cells, self)
            free_cells.state = free_cells.NOT_BOX
            self.grid.place_agent(free_cells, cells)
            self.schedule.add(free_cells)
            #not_boxes.remove(free_cells)
            
            
        not_boxes = list(self.grid.empties)
        """ Una vez repartidas las celdas con caja y sin cajas, Repartimos los robots"""
        for i in range(5):
            x = random.randint(0, height - 1)
            y = random.randint(0, width - 1)

            robot = RobotAgent(i, self, (x,y))
            self.grid.place_agent(robot, (x,y))
            self.schedule.add(robot)
            
        self.datacollector = DataCollector(model_reporters = {'Grid': get_grid})
        
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step() 
        

In [150]:
# INFORMACION DE LA WAREHOUSE:
WIDTH = 10
HEIGHT = 10
NUM_BOX = 40
NUM_ROBOTS = 90

#Tiempo de ejecución
MAX_GENERATIONS = .1
moves = 0

start_time = time.time()
model = WareHouseModel(WIDTH, HEIGHT, NUM_BOX)

while((time.time() - start_time) < MAX_GENERATIONS):
    model.step()
    if(model.cont_boxes != NUM_BOX):
        tiempo_total = time.time() - start_time
        
real_time = str(datetime.timedelta(seconds = (time.time() - start_time)))

In [151]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [152]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='gist_rainbow')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [153]:
print("Verde: stack = 1\nAmarilla: stack = 2\nVerde: stack = 3\nAzul celeste: stack = 4\nNaranja: stack = 5")
print("Azul rey: Cuando el robot está cargando una caja")
print("Rosa: Fondo de pantalla\n\n")
print("--------------------------------RESULTADOS---------------------------------")
print("Numero de movimientos realizados por todos los robots", model.cont_steps)
print("Numero de movimientos realizados por robot", model.cont_steps//5)
print("Cantidad de cajas apiladas", model.cont_boxes )
print("Tiempo total de trabajo para apilar las cajas", tiempo_total )
print("---------------------------------------------------------------------------")
anim

Verde: stack = 1
Amarilla: stack = 2
Verde: stack = 3
Azul celeste: stack = 4
Naranja: stack = 5
Azul rey: Cuando el robot está cargando una caja
Rosa: Fondo de pantalla


--------------------------------RESULTADOS---------------------------------
Numero de movimientos realizados por todos los robots 2081
Numero de movimientos realizados por robot 416
Cantidad de cajas apiladas 40
Tiempo total de trabajo para apilar las cajas 0.09674358367919922
---------------------------------------------------------------------------
